In [1]:
%reload_ext autoreload
%autoreload 2

import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp
import sys
import os

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *

from registration_utilities import *

from skimage.measure import marching_cubes, correct_mesh_orientation

from vis3d_utilities import *
from itertools import izip

%matplotlib inline

Setting environment for Local Macbook Pro


/usr/lib/python2.7/dist-packages/pkg_resources.py:1031: UserWarning: /home/yuncong/.python-eggs is writable by group/others and vulnerable to attack when used with get_resource_filename. Consider a more secure location (set with .set_extraction_path or the PYTHON_EGG_CACHE environment variable).
  warnings.warn(msg, UserWarning)


In [2]:
stack_moving = 'MD594'
stack_fixed = 'MD589'

In [3]:
atlasAlignParams_dir = create_if_not_exists(atlasAlignParams_rootdir + '/%(stack_moving)s_to_%(stack_fixed)s' % \
                     {'stack_moving': stack_moving, 'stack_fixed': stack_fixed})

In [4]:
volume_fixed = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_down32_annotationVolume.bp' % {'stack': stack_fixed}))
volume_moving = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_down32_annotationVolume.bp' % {'stack': stack_moving}))
print volume_fixed.shape
print volume_moving.shape

(348, 421, 377)
(326, 515, 368)


In [5]:
with open(atlasAlignParams_dir + '/%(stack_moving)s_down32_annotationVolume_to_%(stack_fixed)s_down32_annotationVolume_parameters.txt' % \
          {'stack_moving': stack_moving, 'stack_fixed': stack_fixed}, 'r') as f:
    
    lines = f.readlines()
    
    global_params = one_liner_to_arr(lines[0], float)
    centroid_m = one_liner_to_arr(lines[1], float)
    xdim_m, ydim_m, zdim_m  = one_liner_to_arr(lines[2], int)
    centroid_f = one_liner_to_arr(lines[3], float)
    xdim_f, ydim_f, zdim_f  = one_liner_to_arr(lines[4], int)

In [6]:
label_to_name_m, name_to_label_m = DataManager.load_volume_label_to_name(stack_moving)
label_to_name_f, name_to_label_f = DataManager.load_volume_label_to_name(stack_fixed)

In [7]:
colors = {name_u: np.array(random_colors(1)[0])/255.
          for name_u in set([convert_name_to_unsided(name_s) for name_s in name_to_label_m.keys()])}

In [ ]:
# Save meshes of moving volume aligned to fixed

In [8]:
moving_volume_alignedTo_fixed_volume_fn = os.path.join('/home/yuncong/CSHL_atlasAlignParams_atlas_v2/', stack_moving+'_to_'+stack_fixed, 
             "%(stack_moving)s_down32_annotationVolume_alignedTo_%(stack_fixed)s_down32_annotationVolume.bp" %\
             {'stack_moving': stack_moving, 'stack_fixed': stack_fixed})

In [9]:
volume_m_alignedTo_f = bp.unpack_ndarray_file(moving_volume_alignedTo_fixed_volume_fn)

In [10]:
vol_f_xmin, vol_f_xmax, vol_f_ymin, vol_f_ymax, vol_f_zmin, vol_f_zmax = DataManager.load_volume_bbox(stack_fixed)

In [11]:
m_alignedTo_f_meshes_dir = create_if_not_exists(os.path.join('/home/yuncong/CSHL_meshes/', stack_moving+'_to_'+stack_fixed))

In [12]:
for name_s, label in name_to_label_m.iteritems():
    
    xmin, xmax, ymin, ymax, zmin, zmax = bbox_3d(volume_m_alignedTo_f == label)
    vol = (volume_m_alignedTo_f[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == label).astype(np.float)

    t = time.time()
    polydata = volume_to_polydata(vol, origin=(vol_f_xmin+xmin, vol_f_ymin+ymin, vol_f_zmin+zmin), 
                                  num_simplify_iter=3, smooth=True)
    sys.stderr.write('time: %.2f\n' % (time.time() - t) )

    polydata_fn = os.path.join(m_alignedTo_f_meshes_dir, 
                "%(stack_moving)s_down32_annotationVolume_alignedTo_%(stack_fixed)s_down32_annotationVolume_%(name)s_smoothed.stl" %\
                  {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'name': name_s})
    
    save_mesh_stl(polydata, polydata_fn)

marching cube: 0.06 seconds
compute surface area: 0.01 seconds
fill point array: 0.02 seconds
fill cell array: 0.00 seconds
mesh_to_polydata: 0.02 seconds
simplify 0 @ 12931: 0.13 seconds
simplify 1 @ 2587: 0.09 seconds
simplify 2 @ 519: 0.02 seconds
time: 0.33
marching cube: 0.11 seconds
compute surface area: 0.01 seconds
fill point array: 0.04 seconds
fill cell array: 0.00 seconds
mesh_to_polydata: 0.04 seconds
simplify 0 @ 32510: 0.33 seconds
simplify 1 @ 6503: 0.31 seconds
simplify 2 @ 1302: 0.06 seconds
time: 0.86
marching cube: 0.04 seconds
compute surface area: 0.01 seconds


KeyboardInterrupt: 

In [ ]:
# Load moving volume aligned to fixed

In [13]:
moving_volume_alignedTo_fixed_volume_polydata_list = {}

for name_s, label in name_to_label_m.iteritems():
    
    fn = os.path.join(m_alignedTo_f_meshes_dir, 
                      "%(stack_moving)s_down32_annotationVolume_alignedTo_%(stack_fixed)s_down32_annotationVolume_%(name)s_smoothed.stl" %\
                      {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'name': name_s})

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()
    moving_volume_alignedTo_fixed_volume_polydata_list[name_s] = polydata

In [14]:
# Load moving volume

In [15]:
moving_volume_polydata_list = {}

for name_s, label in name_to_label_m.iteritems():
    
    fn = os.path.join(mesh_rootdir, stack_moving, "%(stack)s_down32_annotationVolume_%(name)s_smoothed.stl" % {'stack': stack_moving, 'name': name_s})

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()
    moving_volume_polydata_list[name_s] = polydata

In [16]:
# Load fixed volume

In [17]:
fixed_volume_polydata_list = {}

for name_s, label in name_to_label_f.iteritems():
    
    fn = os.path.join(mesh_rootdir, stack_fixed, "%(stack)s_down32_annotationVolume_%(name)s_smoothed.stl" % {'stack': stack_fixed, 'name': name_s})

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()
    fixed_volume_polydata_list[name_s] = polydata

In [ ]:
# Show each structure in one color

volume_m_alignedTo_f_mesh_actors = [actor_mesh(polydata, colors[convert_name_to_unsided(name_s)], wireframe=True, opacity=1.) 
               for name_s, polydata in moving_volume_alignedTo_fixed_volume_polydata_list.iteritems()]

volume_f_mesh_actors = [actor_mesh(polydata, colors[convert_name_to_unsided(name_s)], wireframe=True, opacity=1.) 
               for name_s, polydata in fixed_volume_polydata_list.iteritems()]

launch_vtk(volume_m_alignedTo_f_mesh_actors + volume_f_mesh_actors + [outerContour_actor])

In [ ]:
# Show each brain in one color

volume_m_alignedTo_f_mesh_actors = [actor_mesh(polydata, (1.,0,0), wireframe=True, opacity=1.) 
               for name_s, polydata in moving_volume_alignedTo_fixed_volume_polydata_list.iteritems()]

volume_f_mesh_actors = [actor_mesh(polydata, (0,0,1), wireframe=True, opacity=1.) 
               for name_s, polydata in fixed_volume_polydata_list.iteritems()]

volume_m_mesh_actors = [actor_mesh(polydata, (0,1,1), wireframe=True, opacity=1.) 
               for name_s, polydata in moving_volume_polydata_list.iteritems()]

launch_vtk(volume_m_alignedTo_f_mesh_actors + volume_f_mesh_actors + volume_m_mesh_actors)

In [ ]:
launch_vtk(volume_m_alignedTo_f_mesh_actors + volume_f_mesh_actors)

In [18]:
centroid_allLandmarks = defaultdict(list)
polydata_list_allLandmarks = {}
average_polydata_allLandmarks = {}
average_volume_allLandmarks = {}
average_volume_origin_allLandmarks = {}

atlas_space_structure_volumes = {}
atlas_space_structure_meshes = {}

In [19]:
all_names = set([convert_name_to_unsided(name_s) for name_s in name_to_label_f.keys()])

In [20]:
for name_u in all_names:

    print name_u
    
    vertices_list = []
    faces_list = []
    
    if name_u in singular_structures:
        
        fn = os.path.join(mesh_rootdir, stack_fixed, "%(stack)s_down32_annotationVolume_%(name)s_smoothed.stl" % \
                              {'stack': stack_fixed, 'name': name_u})

        vertices, faces = load_mesh_stl(fn)

        vertices_mirrored = vertices.copy()
        zmean = vertices[:,2].mean(axis=0)
        vertices_mirrored[:, 2] = - (vertices[:, 2] - zmean) + zmean # mirror L to align with R, keep zmin        

        vertices_list.append(vertices)
        faces_list.append(faces)

        vertices_list.append(vertices_mirrored)
        faces_list.append(faces)
        
        centroid = vertices.mean(axis=0)
        centroid_allLandmarks[name_u].append(centroid)

        centroid_anchor = centroid.copy()
        
    
    else:
    
        lname = convert_to_left_name(name_u)
        rname = convert_to_right_name(name_u)

        if lname in name_to_label_f:

            ############## Left #############

            fn = os.path.join(mesh_rootdir, stack_fixed, "%(stack)s_down32_annotationVolume_%(name)s_smoothed.stl" % \
                              {'stack': stack_fixed, 'name': lname})

            vertices, faces = load_mesh_stl(fn)

            zmean = vertices[:,2].mean(axis=0)
            vertices[:, 2] = - (vertices[:, 2] - zmean) + zmean # mirror L to align with R, keep zmin

            vertices_list.append(vertices)
            faces_list.append(faces)

            centroid = vertices.mean(axis=0)
            centroid_allLandmarks[lname].append(centroid)

            centroid_anchor = centroid.copy()


        if rname in name_to_label_f:

            ################# Right ##############  


            fn = os.path.join(mesh_rootdir, stack_fixed, "%(stack)s_down32_annotationVolume_%(name)s_smoothed.stl" % \
                              {'stack': stack_fixed, 'name': rname})

            vertices, faces = load_mesh_stl(fn)

            vertices_list.append(vertices)
            faces_list.append(faces)

            centroid = vertices.mean(axis=0)
            centroid_allLandmarks[rname].append(centroid)


        if lname in name_to_label_m:

            fn = os.path.join(m_alignedTo_f_meshes_dir, 
                          "%(stack_moving)s_down32_annotationVolume_alignedTo_%(stack_fixed)s_down32_annotationVolume_%(name)s_smoothed.stl" %\
                          {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'name': lname})

            vertices, faces = load_mesh_stl(fn)

            zmean = vertices[:,2].mean(axis=0)
            vertices[:, 2] = - (vertices[:, 2] - zmean) + zmean # mirror L to align with R, keep zmin

            vertices_list.append(vertices)
            faces_list.append(faces)


            centroid = vertices.mean(axis=0)
            centroid_allLandmarks[lname].append(centroid)

        if rname in name_to_label_m:

            fn = os.path.join(m_alignedTo_f_meshes_dir, 
                          "%(stack_moving)s_down32_annotationVolume_alignedTo_%(stack_fixed)s_down32_annotationVolume_%(name)s_smoothed.stl" %\
                          {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'name': rname})
            vertices, faces = load_mesh_stl(fn)

            vertices_list.append(vertices)
            faces_list.append(faces)

            centroid = vertices.mean(axis=0)
            centroid_allLandmarks[rname].append(centroid)
        
        
    ##############################
    
    vertices_aligned_list = [] # all meshes in this list have vertices centered at (0,0)

    for i, vertices in enumerate(vertices_list):

        if i == 0:
            vertices1 = vertices.copy()
            vertices_aligned_list.append(vertices1 - vertices1.mean(axis=0))
        else:            
            t = time.time()                    
            vertices_alignedTo1 = icp(fixed_pts=vertices1, moving_pts=vertices, num_iter=100, rotation_only=True)
            sys.stderr.write('icp: %.2f\n' % (time.time() - t))

            vertices_aligned_list.append(vertices_alignedTo1 - vertices_alignedTo1.mean(axis=0))

    ################################
    
    polydata_list = [mesh_to_polydata(vs, fs) for vs, fs in zip(vertices_aligned_list, faces_list)]
    
    polydata_list_allLandmarks[name_u] = polydata_list
        
    average_volume, average_volume_origin, average_polydata = average_shape(polydata_list, num_simplify_iter=3, smooth=True)
    # these all have centroid at (0,0,0)
    
    average_polydata_allLandmarks[name_u] = average_polydata
    average_volume_allLandmarks[name_u] = average_volume
    average_volume_origin_allLandmarks[name_u] = average_volume_origin
    
    
#     instance_actors = [actor_mesh(polydata, np.random.randint(0,255,3)/255., opacity=.5, wireframe=True) 
#                    for polydata in polydata_list_allLandmarks[name_u]]

#     average_mesh_actor = actor_mesh(average_polydata_allLandmarks[name_u], (1,1,1), opacity=.5, wireframe=True)
#     launch_vtk(instance_actors + [average_mesh_actor])
    
#     average_volume_actor = actor_volume(average_volume.astype(np.float32), 
#                                         what='probability', origin=average_volume_origin)
#     launch_vtk(instance_actors + [average_volume_actor])

icp: 2.41
icp: 2.28
icp: 2.76
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
polydata_to_volume: 0.00
polydata_to_volume: 0.00
polydata_to_volume: 0.00
polydata_to_volume: 0.00
find common: 0.00
marching cube: 0.02 seconds
compute surface area: 0.00 seconds
fill point array: 0.01 seconds
fill cell array: 0.00 seconds
mesh_to_polydata: 0.01 seconds
simplify 0 @ 4400: 0.05 seconds


Tz
VCA

simplify 1 @ 881: 0.06 seconds
volume_to_polydata: 0.16
icp: 9.27
icp: 2.97
icp: 8.50
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
polydata_to_volume: 0.00
polydata_to_volume: 0.00
polydata_to_volume: 0.00
polydata_to_volume: 0.00
find common: 0.01
marching cube: 0.05 seconds
compute surface area: 0.00 seconds
fill point array: 0.02 seconds
fill cell array: 0.00 seconds
mesh_to_polydata: 0.02 seconds
simplify 0 @ 12091: 0.11 seconds



7n

simplify 1 @ 2420: 0.11 seconds
simplify 2 @ 485: 0.02 seconds
volume_to_polydata: 0.31
icp: 5.24
icp: 5.79
icp: 5.59
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
polydata_to_volume: 0.00
polydata_to_volume: 0.01
polydata_to_volume: 0.01
polydata_to_volume: 0.02
find common: 0.01
marching cube: 0.06 seconds
compute surface area: 0.00 seconds
fill point array: 0.01 seconds
fill cell array: 0.00 seconds
mesh_to_polydata: 0.01 seconds
simplify 0 @ 8982: 0.06 seconds


...


SC

simplify 1 @ 5713: 0.24 seconds
simplify 2 @ 1144: 0.05 seconds
volume_to_polydata: 0.65
icp: 134.12
fill point array: 0.01 seconds
fill cell array: 0.00 seconds
fill point array: 0.01 seconds
fill cell array: 0.00 seconds
polydata_to_volume: 0.09
polydata_to_volume: 0.10
find common: 0.09
marching cube: 0.63 seconds
compute surface area: 0.02 seconds
fill point array: 0.12 seconds
fill cell array: 0.00 seconds
mesh_to_polydata: 0.12 seconds
simplify 0 @ 96444: 0.86 seconds
simplify 1 @ 19290: 0.96 seconds
simplify 2 @ 3859: 0.19 seconds



RtTg

volume_to_polydata: 2.81
icp: 26.05
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
polydata_to_volume: 0.01
polydata_to_volume: 0.02
find common: 0.01
marching cube: 0.12 seconds
compute surface area: 0.01 seconds
fill point array: 0.03 seconds
fill cell array: 0.00 seconds
mesh_to_polydata: 0.03 seconds
simplify 0 @ 21188: 0.19 seconds


simplify 1 @ 4241: 0.17 seconds
simplify 2 @ 851: 0.03 seconds
volume_to_polydata: 0.56


In [46]:
for i, polydata in enumerate(polydata_list_allLandmarks['7N']):
    instance_actors = actor_mesh(polydata, [(1,0,0),(0,1,0),(0,0,1),(1,1,0)][i], opacity=1, wireframe=False)
    launch_vtk([instance_actors], snapshot_fn='/home/yuncong/7N_%d.png' % i, interactive=True, init_angle='coronal')

In [44]:
instance_actors = [actor_mesh(polydata, [(1,0,0),(0,1,0),(0,0,1),(1,1,0)][i], opacity=.5, wireframe=True) 
                   for i, polydata in enumerate(polydata_list_allLandmarks['7N'])]
launch_vtk(instance_actors, snapshot_fn='/home/yuncong/7N_aligned.png', interactive=False, init_angle='coronal')

In [45]:
average_volume_actor = actor_volume(average_volume_allLandmarks['7N'].astype(np.float32), 
                                        what='probability', origin=average_volume_origin_allLandmarks['7N'])
launch_vtk([average_volume_actor], snapshot_fn='/home/yuncong/7N_prob.png', interactive=False, init_angle='coronal')

In [49]:
actor = actor_mesh(average_polydata_allLandmarks['7N'], [1,1,1], opacity=.5, wireframe=True)
launch_vtk([actor], snapshot_fn='/home/yuncong/7N_thresholded.png', interactive=False, init_angle='coronal')

In [50]:
canonical_locations, canonical_centroid, canonical_normal = average_location(centroid_allLandmarks)

[ 0.03550233 -0.09604324  0.99474382] @ [ 796.99383545  373.38763428  308.71014404]


In [51]:
for name_u in all_names:
    
    if name_u in singular_structures:
        centroid = canonical_locations[name_u]

        average_volume = average_volume_allLandmarks[name_u]
        average_volume_origin = average_volume_origin_allLandmarks[name_u]
        average_polydata = average_polydata_allLandmarks[name_u]

        atlas_space_structure_volumes[name_u] = (average_volume, 
                                            centroid + average_volume_origin)
        
        vs, fs = polydata_to_mesh(average_polydata)
        atlas_space_structure_meshes[name_u] = mesh_to_polydata(vs + centroid, fs)
        
    else:
    
        lname = convert_to_left_name(name_u)
        rname = convert_to_right_name(name_u)

        centroid_r = canonical_locations[rname]
        centroid_l = canonical_locations[lname]

        average_volume = average_volume_allLandmarks[name_u]
        average_volume_origin = average_volume_origin_allLandmarks[name_u]
        average_polydata = average_polydata_allLandmarks[name_u]

        atlas_space_structure_volumes[rname] = (average_volume, 
                                            centroid_r + average_volume_origin)
        atlas_space_structure_volumes[lname] = mirror_volume(average_volume, 
                                             centroid_l + average_volume_origin*(1,1,-1))

        vs, fs = polydata_to_mesh(average_polydata)
        atlas_space_structure_meshes[rname] = mesh_to_polydata(vs + centroid_r, fs)
        atlas_space_structure_meshes[lname] = mesh_to_polydata(vs*(1,1,-1) + centroid_l, fs)

fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 s

In [52]:
all_atlas_structure_volume_actors = []

for name_u in all_names:
    
    if name_u in singular_structures:

        volume, origin = atlas_space_structure_volumes[name_u]
        structure_volume_actor = actor_volume(volume.astype(np.float32), what='probability', origin=origin, c=colors[name_u])

        all_atlas_structure_volume_actors.append(structure_volume_actor)

    else:
    
        lname = convert_to_left_name(name_u)
        volume, origin = atlas_space_structure_volumes[lname]
        structure_volume_actor = actor_volume(volume.astype(np.float32), what='probability', origin=origin, c=colors[name_u])

        all_atlas_structure_volume_actors.append(structure_volume_actor)

        rname = convert_to_right_name(name_u)
        volume, origin = atlas_space_structure_volumes[rname]
        structure_volume_actor = actor_volume(volume.astype(np.float32), what='probability', origin=origin, c=colors[name_u])

        all_atlas_structure_volume_actors.append(structure_volume_actor)

In [53]:
launch_vtk(all_atlas_structure_volume_actors, init_angle='sagittal', background_color=(0,0,0))

In [54]:
for name_s, p in atlas_space_structure_meshes.iteritems():
    save_mesh_stl(p, os.path.join(mesh_rootdir, 'atlas', '%s.stl'%name_s))

In [55]:
all_atlas_structure_mesh_actors = []

for name_u in all_names:
    
    if name_u in singular_structures:
        
        structure_mesh_actor = actor_mesh(atlas_space_structure_meshes[name_u], color=colors[name_u])
        all_atlas_structure_mesh_actors.append(structure_mesh_actor)
        
    else:
    
        lname = convert_to_left_name(name_u)
        structure_mesh_actor = actor_mesh(atlas_space_structure_meshes[lname], color=colors[name_u])
        all_atlas_structure_mesh_actors.append(structure_mesh_actor)

        rname = convert_to_right_name(name_u)
        structure_mesh_actor = actor_mesh(atlas_space_structure_meshes[rname], color=colors[name_u])
        all_atlas_structure_mesh_actors.append(structure_mesh_actor)

In [56]:
def move_polydata(polydata, d):
    vs, fs = polydata_to_mesh(polydata)
    return mesh_to_polydata(vs + d, fs)

In [57]:
launch_vtk(all_atlas_structure_mesh_actors, init_angle='sagittal')

In [ ]:
launch_vtk(all_atlas_structure_mesh_actors + all_atlas_structure_volume_actors, init_angle='sagittal')

In [58]:
fn = os.path.join(mesh_rootdir, stack_fixed, "%(stack)s_down32_outerContourVolume_smoothed.stl" % {'stack': stack_fixed})

reader = vtk.vtkSTLReader()
reader.SetFileName(fn)
reader.Update()

outerContour_polydata = reader.GetOutput()

outerContour_polydata = move_polydata(outerContour_polydata, -canonical_centroid)

outerContour_actor = actor_mesh(outerContour_polydata, (1,1,1), opacity=.5, wireframe=False)

fill point array: 0.05 seconds
fill cell array: 0.00 seconds


In [ ]:
launch_vtk(all_atlas_structure_mesh_actors + [outerContour_actor], init_angle='sagittal')

In [ ]:
launch_vtk(all_atlas_structure_volume_actors + [outerContour_actor], init_angle='sagittal')

In [ ]:
# from collections import defaultdict
# centroid_allLandmarks = defaultdict(list)
# average_polydata_allLandmarks = {}
# polydata_list_allLandmarks = {}
# average_volume_allLandmarks = {}

# # for name_unsided in labels_unsided:
# # # for name_unsided in ['outerContour']:
# # # for name_unsided in ['Pn']:
    
# #     if name_unsided == 'outerContour' or name_unsided == 'sp5':
# #         continue
        
# #     names = labelMap_unsidedToSided[name_unsided]
# #     print name_unsided, names

# for name in names:
    
#     vertices_list = []
#     faces_list = []
    
#     for stack in ['MD589', 'MD594']:
# #     for stack in ['MD594']:

#         if len(names) == 2:
        
#             for name_sided in names:

#                 fn = "/home/yuncong/CSHL_meshes/%(stack)s/%(stack)s_%(label)s_gaussianSmoothed.stl" % {'stack':stack, 'label':name_sided}
               
#                 if os.path.exists(fn):
#                     vertices, faces = load_mesh_stl(fn)
#                 else:
#                     continue
            
#                 if '_L' in name_sided:
#                     zmean = vertices[:,2].mean(axis=0)
#                     vertices[:, 2] = - (vertices[:, 2] - zmean) + zmean # mirror L to align with R, keep zmin        

#                 if stack == 'MD594':
#                     vertices_alignedToScoreVolume = vertices + (ann_xmin,ann_ymin,ann_zmin) - (sco_xmin,sco_ymin,sco_zmin)
#                     vertices = transform_points_inverse(global_params, pts_prime=vertices_alignedToScoreVolume, 
#                                                                        c_prime=test_centroid, c=atlas_centroid)
                    
#                 vertices_list.append(vertices)    
#                 faces_list.append(faces)
                
#                 centroid_prime = vertices.mean(axis=0)

#                 if stack == 'MD594':

#                     centroid_prime_alignedToScoreVolume = centroid_prime + (ann_xmin,ann_ymin,ann_zmin) - (sco_xmin,sco_ymin,sco_zmin)
#                     centroid_prime_alignedToAtlas = transform_points_inverse(global_params, pts_prime=[centroid_prime_alignedToScoreVolume], 
#                                                                        c_prime=test_centroid, c=atlas_centroid)[0]
#                     centroid_allLandmarks[name_sided].append(centroid_prime_alignedToAtlas)
#                 else:
#                     centroid_allLandmarks[name_sided].append(centroid_prime)
        
#         elif len(names) == 1:
            
#             name_sided = names[0]
            
#             fn = "/home/yuncong/CSHL_meshes/%(stack)s/%(stack)s_%(label)s_gaussianSmoothed.stl" % {'stack':stack, 'label':name_sided}
            
#             if os.path.exists(fn):
#                 vertices, faces = load_mesh_stl(fn)
#             else:
#                 continue

#             vertices_mirrored = vertices.copy()    
#             zmean = vertices[:,2].mean(axis=0)
#             vertices_mirrored[:, 2] = - (vertices[:, 2] - zmean) + zmean # mirror L to align with R, keep zmin        
            
#             t = time.time()
            
#             if name_unsided == 'sp5' or name_unsided == 'outerContour':
# #                 _, symmetric_poly = average_shape([mesh_to_polydata(vertices, faces), mesh_to_polydata(vertices_mirrored, faces)],
# #                                               num_simplify_iter=5, smooth=True)
#                 symmetric_poly = mesh_to_polydata(vertices, faces)
#             else:
#                 _, symmetric_poly = average_shape([mesh_to_polydata(vertices, faces), mesh_to_polydata(vertices_mirrored, faces)],
#                                               num_simplify_iter=3, smooth=True)
                
#             # must simplify so the ICP later can be fast
#             # must also smooth, because otherwise the number of points gets stuck
            
#             sys.stderr.write('average shape: %.2f\n' % (time.time() - t))
#             # cost mostly comes from :
#             # - marching cube
#             # - first two rounds of decimation and smoothing

#             symmetric_vertices, symmetric_faces = polydata_to_mesh(symmetric_poly)
         
#             if stack == 'MD594':
#                 vertices_alignedToScoreVolume = symmetric_vertices + (ann_xmin,ann_ymin,ann_zmin) - (sco_xmin,sco_ymin,sco_zmin)
#                 symmetric_vertices = transform_points_inverse(global_params, pts_prime=vertices_alignedToScoreVolume, 
#                                                                    c_prime=test_centroid, c=atlas_centroid)
        
#             vertices_list.append(symmetric_vertices)
#             faces_list.append(symmetric_faces)
            
#             centroid_prime = symmetric_vertices.mean(axis=0)

#             if stack == 'MD594':

#                 centroid_prime_alignedToScoreVolume = centroid_prime + (ann_xmin,ann_ymin,ann_zmin) - (sco_xmin,sco_ymin,sco_zmin)
#                 centroid_prime_alignedToAtlas = transform_points_inverse(global_params, pts_prime=[centroid_prime_alignedToScoreVolume], 
#                                                                    c_prime=test_centroid, c=atlas_centroid)[0]
#                 centroid_allLandmarks[name_sided].append(centroid_prime_alignedToAtlas)
#             else:
#                 centroid_allLandmarks[name_sided].append(centroid_prime)
    
#     ###### Align meshes ######
    
#     vertices_aligned_list = []

#     for i, vertices in enumerate(vertices_list):

# #         if i == 0:
# #             vertices1 = vertices.copy()
# #             vol_fixed, origin_fixed, _ = polydata_to_volume(mesh_to_polydata(vertices, faces_list[0]))
# #             vertices_aligned_list.append(vertices1)
# #         else:            
# #             t = time.time()
# #             vol_moving, origin_moving, _ = polydata_to_volume(mesh_to_polydata(vertices, faces_list[i]))
# #             T, c_fixed, c_moving = optimize_rigid_lieGroup(vol_fixed, vol_moving)
# #             vertices_alignedTo1 = transform_points(T, vertices-origin_moving, c=c_moving, c_prime=c_fixed) + origin_fixed            
# #             sys.stderr.write('align: %.2f\n' % (time.time() - t))

# #             vertices_aligned_list.append(vertices_alignedTo1)
                
#         if i == 0:
#             vertices1 = vertices.copy()
#             vertices_aligned_list.append(vertices1)
#         else:            
#             t = time.time()                    
#             vertices_alignedTo1 = icp(fixed_pts=vertices1, moving_pts=vertices, num_iter=100)
#             sys.stderr.write('icp: %.2f\n' % (time.time() - t))

#             vertices_aligned_list.append(vertices_alignedTo1)
    
#     polydata_list = [mesh_to_polydata(vs, fs) for vs, fs in zip(vertices_aligned_list, faces_list)]
            
#     # if landmark has only one instance, add its mirrored version
#     if len(names) == 1:
#         for vs, fs in zip(vertices_aligned_list, faces_list):
#             zmean = vs[:,2].mean(axis=0)
#             vs_mirrored = vs.copy()
#             vs_mirrored[:, 2] = - (vs[:, 2] - zmean) + zmean # mirror L to align with R, keep zmin        
#             polydata_list.append(mesh_to_polydata(vs_mirrored, fs))

#     polydata_list_allLandmarks[name_unsided] = polydata_list
    
# #     ######### Compute Average #########
#     t = time.time()

#     if name_unsided == 'outerContour' or name_unsided == 'sp5':
#         # since alignment is not ideal, just use MD589's outerContour for now
# #         average_volume, average_polydata = average_shape([polydata_list[0],polydata_list[2]], num_simplify_iter=5, smooth=True, concensus_percentage=0.9)
# #         average_volume, average_polydata = average_shape([polydata_list[0],polydata_list[0]], num_simplify_iter=5, smooth=False, concensus_percentage=0.9)
#         average_volume = None
#         average_polydata = polydata_list[2]
#     elif len(names) == 1:
#         average_volume, average_polydata = average_shape(polydata_list, num_simplify_iter=3, smooth=True, concensus_percentage=0.9)
#     else:
#         average_volume, average_polydata = average_shape(polydata_list, num_simplify_iter=3, smooth=True)
    
#     sys.stderr.write('average shape: %.2f\n' % (time.time() - t))

#     average_polydata_allLandmarks[name_unsided] = average_polydata

#     average_volume_allLandmarks[name_unsided] = average_volume
    
# #     bp.pack_ndarray_file(average_volume, mesh_rootdir + "/%(name)s_average.bp" % {'stack': stack, 'name': name_unsided})
# #     save_mesh_stl(average_polydata, mesh_rootdir + "/%(name)s_average.stl" % {'stack': stack, 'name': name_unsided})

# centroid_allLandmarks.default_factory = None